<h2>Implementing a Convolutional Neural Network with Keras CIFAR dataset</h2>

In [10]:
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from matplotlib import pyplot as plt
from keras.utils import np_utils 
import numpy as np

<h3>Reading the dataset</h3>

In [8]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 32s 0us/step


<h3>Pre processing the dataset</h3>

In [26]:
num_train, height, width, depth = X_train.shape
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0]

#normalizing the data

#error :No loop matching the specified signature and casting was found for ufunc true_divide
#convert the data into float

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_test /= np.max(X_test)
X_train /= np.max(X_train)

#one hot encode the labels
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

In [27]:
X_train[0][0][0] #these values indicate a pixel of the data after normalization

array([0.23137255, 0.24313726, 0.24705882], dtype=float32)

In [29]:
y_train[0] #and the label for X[0] is 6

array([6], dtype=uint8)

<h3>Setting the parameters</h3>

In [31]:
batch_size = 100
num_epochs = 10
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

<h3>Creating the model and Fitting the model</h3>

In [32]:
inp = Input(shape=(height, width, depth))
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)

conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)

flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)

out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out)

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1)

model.evaluate(X_test, Y_test, verbose=1) 


Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 51s 1ms/step - loss: 1.5921 - acc: 0.4177 - val_loss: 1.2321 - val_acc: 0.5634
Epoch 2/10
45000/45000 [==============================] - 49s 1ms/step - loss: 1.1926 - acc: 0.5739 - val_loss: 0.9765 - val_acc: 0.6554
Epoch 3/10
45000/45000 [==============================] - 48s 1ms/step - loss: 1.0190 - acc: 0.6389 - val_loss: 0.8825 - val_acc: 0.6968
Epoch 4/10
45000/45000 [==============================] - 48s 1ms/step - loss: 0.9080 - acc: 0.6792 - val_loss: 0.8330 - val_acc: 0.7160
Epoch 5/10
45000/45000 [==============================] - 49s 1ms/step - loss: 0.8322 - acc: 0.7064 - val_loss: 0.7665 - val_acc: 0.7334
Epoch 6/10
45000/45000 [==============================] - 51s 1ms/step - loss: 0.7712 - acc: 0.7265 - val_loss: 0.7064 - val_acc: 0.7602
Epoch 7/10
45000/45000 [==============================] - 51s 1ms/step - loss: 0.7176 - acc: 0.7473 - val_loss: 0.6898 - val_acc:

[0.6779823116779328, 0.7668]